### 1. Generating data...

In [ ]:
from ecog2vec.data_generator import NeuralDataGenerator

# efc400 = NeuralDataGenerator('/NWB/EFC400', 'EFC400')
# efc400.bad_electrodes = [1, 2, 33, 50, 54, 64, 128, 129, 193, 194, 256]
# efc400.bad_electrodes = [x - 1 for x in efc400.bad_electrodes]
# efc400.good_electrodes = [x for x in efc400.good_electrodes if x not in efc400.bad_electrodes]
# efc400.electrode_name = 'R256GridElectrode electrodes'

# efc401 = NeuralDataGenerator('/NWB/EFC401', 'EFC401')
# efc401.bad_electrodes = [1,2,63,64,65,127,143,193,194,195,196,235,239,243,252,254,255,256]
# efc401.bad_electrodes = [x - 1 for x in efc401.bad_electrodes]
# efc401.good_electrodes = [x for x in efc401.good_electrodes if x not in efc401.bad_electrodes]
# efc401.electrode_name = # Not needed, default value set to 401's electrode name (L256GridElectrode electrodes)

# efc402 = NeuralDataGenerator('/NWB/EFC402', 'EFC402')
# efc402.bad_electrodes = [129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 
#                          139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 
#                          149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 
#                          159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 
#                          169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 
#                          179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 
#                          189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 
#                          199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 
#                          209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 
#                          219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 
#                          229, 230, 231, 232, 233, 234, 235, 236, 237, 238,
#                          239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 
#                          249, 250, 251, 252, 253, 254, 255, 256]
# efc402.bad_electrodes = [x - 1 for x in efc402.bad_electrodes]
# efc402.good_electrodes = [x for x in efc402.good_electrodes if x not in efc402.bad_electrodes]
# efc402.electrode_name = 'InferiorGrid electrodes'
# efc402._bipolar_to_elec_map = efc402.bipolar_to_elec_map()
# efc402._good_channels = efc402.good_channels()

efc403 = NeuralDataGenerator('/NWB/EFC403', 'EFC403')
efc403.bad_electrodes = [129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
                         139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
                         149, 161, 162, 163, 164, 165, 166, 167, 168, 169,
                         170, 171, 172, 173, 174, 175, 176, 177, 178, 179,
                         180, 181]
efc403.bad_electrodes = [x - 1 for x in efc403.bad_electrodes]
efc403.good_electrodes = [x for x in efc403.good_electrodes if x not in efc403.bad_electrodes]
efc403.electrode_name = 'Grid electrodes'
# efc403._bipolar_to_elec_map = efc403.bipolar_to_elec_map()
# efc403._good_channels = efc403.good_channels()

print(len(efc403.good_electrodes))

In [ ]:
import os
# Save clipped data for training,
# save unclipped data to extract features from,
# save entire recording as chunks for training,
# save entire reocording to extract features from
parent_dir = '/home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs'

chopped_sentence_dir = '/home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs/chopped_sentence'
sentence_dir = '/home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs/sentence'
chopped_recording_dir = '/home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs/chopped_recording'
full_recording_dir = '/home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs/full_recording'

for directory in [parent_dir, chopped_sentence_dir, sentence_dir, chopped_recording_dir, full_recording_dir]:
    os.makedirs(directory, exist_ok=True)
    print(f"Directory created or already exists: {directory}")

chunk_length = 200000

block_list = []

# 'write_raw_data', but really it's filtered to the high
# gamma range and the analytic amplitude is calculated
efc403.write_raw_data(
    # chopped_sentence_dir=chopped_sentence_dir, # don't need this for now
    # sentence_dir=sentence_dir, # don't need this for now
    full_recording_dir=full_recording_dir, # saves the entire recording, so features can be extracted
    chopped_recording_dir=chopped_recording_dir, # splits recording into chunks for training
    chunk_length=chunk_length,
    BPR=False)  # chunk length... 200k should be fine

### 2. Train and extract `wav2vec` features

Split into test/valid and train.

Strides should downsample by ~30x; `wav2vec` takes 16kHz inputs and our raw data is at 3kHz

In [ ]:
# From facebookresearch/fairseq
# Usage: python3 wav2vec_manifest.py W2V_TRAIN_DATA_PATH W2V_MANIFEST_PATH

!python3 /home/bayuan/Documents/fall23/fairseq/examples/wav2vec/wav2vec_manifest.py \
    /home/bayuan/Documents/fall23/ecog2vec/elu_models/403/wav2vec_inputs_bpr_mocha/chopped_recording \
  --dest /home/bayuan/Documents/fall23/ecog2vec/elu_models/403/manifests_bpr/manifest_o4 \
  --ext wav \
  --valid-percent 0.1 \
  --seed 42

In [ ]:
# Usage from facebookresearch/fairseq

!python3 -c 'import argparse; print(argparse.__file__)'
!python3 /home/bayuan/Documents/fall23/fairseq/train.py    \
  /home/bayuan/Documents/fall23/ecog2vec/elu_models/400/manifest  \
  --save-dir /home/bayuan/Documents/fall23/ecog2vec/elu_tl_models/403/model_400_401_402  \
  --num-workers 6 --fp16 --max-update 400000 --save-interval 1 --infonce --no-epoch-checkpoints   \
  --arch wav2vec --task audio_pretraining --min-lr 1e-06 --stop-min-lr 1e-09 --optimizer adam \
  --lr 0.0001 --lr-scheduler cosine   \
  --conv-feature-layers "[(512, 8, 5), (512, 4, 3), (512, 2, 2)]"   \
  --conv-aggregator-layers "[(512, 2, 1), (512, 3, 1), (512, 4, 1)]"   \
  --skip-connections-agg --residual-scale 0.5 --log-compression --warmup-updates 500 --warmup-init-lr 1e-07 \
  --num-negatives 10   --max-sample-size 1500000 --skip-invalid-size-inputs-valid-test  --batch-size 10 \
  --max-tokens 1500000   --prediction-steps 12 \
  --criterion wav2vec --activation elu --patience 100

#### Extract $c$ embeddings.

extract entire recording features

In [ ]:
# Borrowed from facebookresearch/fairseq with minor modifications

import torch
import fairseq
# from scipy.io import wavfile
import os
import soundfile as sf
import numpy as np
from pynwb import NWBHDF5IO
import pandas as pd

file_list = []
entire_recording_dir = '/home/bayuan/Documents/fall23/ecog2vec/wav2vec_outputs/full_recording' 
saved_latent_sentences_dir = '/home/bayuan/Documents/fall23/ecog2vec/wav2vec_outputs/latent_sentence'
patient = 'EFC402' # CHANGE PER SUBJECT
nwb_file_dir = f'/NWB/{patient}'
nwb_file_prefix = f'{patient}_B'
nwb_file_suffix = f'.wav.pt'

cp_path = '/home/bayuan/Documents/fall23/ecog2vec/elu_tl_models/tl_401_402/checkpoint_best.pt' # CHANGE PER MODEL
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_path])
model = model[0]
model.eval()

for directory in [entire_recording_dir, saved_latent_sentences_dir]:
    os.makedirs(directory, exist_ok=True)
    print(f"Directory created or already exists: {directory}")

dir_path = f'/home/bayuan/Documents/fall23/ecog2vec/elu_models/402/wav2vec_inputs_bpr/full_recording' # CHANGE PER SUBJECT

for file in os.listdir(dir_path):

  wav_path = str(os.path.join(dir_path, file))

  wav_input_16khz, sr = sf.read(wav_path)
  
  
  wav_input_16khz = wav_input_16khz.T
  wav_input_16khz = wav_input_16khz.reshape(1, 128, -1) # CHANGE TO NUMBER OF ELECTRODES

  wav_input_16khz = torch.from_numpy(wav_input_16khz).to(torch.float)
  # print(wav_input_16khz.shape)

  # print(sr, wav_input_16khz.shape)
  z = model.feature_extractor(wav_input_16khz)
  c = model.feature_aggregator(z)
  
  torch.save(c, f"/home/bayuan/Documents/fall23/ecog2vec/wav2vec_outputs/full_recording/{file}.pt")

# Split the recording's context vector into its 
# individual sentences.

for file in os.listdir(entire_recording_dir):
  c_filepath = os.path.join(entire_recording_dir, file)
  
  c = torch.load(c_filepath).detach().numpy()[0]
  nwb_file = file.rstrip(nwb_file_suffix)
  
  nwb_path = os.path.join(nwb_file_dir, nwb_file)
  io = NWBHDF5IO(nwb_path, load_namespaces=True, mode='r')
  nwbfile = io.read()
  
  sr = 101.7
  
  starts = list(nwbfile.trials[:]['start_time'] * sr)
  stops = list(nwbfile.trials[:]['stop_time'] * sr)
  starts = [int(start) for start in starts]
  stops = [int(stop) for stop in stops]
  
  i = 0
  for start, stop in zip(starts, stops):
    speaking_segment = c[:,start:stop]
    torch.save(speaking_segment, f'{saved_latent_sentences_dir}/{nwb_file}_{i}.wav.pt')
    i = i + 1
    
# file_list

Let's look at the data


In [ ]:
import torch
import matplotlib.pyplot as plt

# We don't want periodicity here

c = torch.load('/home/bayuan/Documents/fall23/ecog2vec/wav2vec_outputs/latent_sentence/EFC401_B41.nwb_0.wav.pt')
cc = c# .detach().numpy()[0]

print(cc.shape)
# print(cc[6])

fig, ax = plt.subplots(3,3)

# print(len(cc[0]))
ax[0,0].plot(cc[0])
ax[0,1].plot(cc[1])
ax[0,2].plot(cc[2])
ax[1,0].plot(cc[3])
ax[1,1].plot(cc[4])
ax[1,2].plot(cc[5])
ax[2,0].plot(cc[6])
ax[2,1].plot(cc[7])
ax[2,2].plot(cc[8])

### 3. Write to tf_records for `ecog2txt`

In [ ]:
# Taken from `jgm/ecog2txt`. Minor modifications.

import tensorflow as tf
import os
from machine_learning.neural_networks import tf_helpers as tfh
import pdb

from pynwb import NWBHDF5IO
import numpy as np
import os
import torch
import soundfile as sf

# Creating a list of dictionaries of:
# 
# `ecog_sequence`: ECoG data, clipped to token(-sequence) length
# `text_sequence`: the corresponding text token(-sequence)
# `audio_sequence`: the corresponding audio (MFCC) token sequence (gonna set to)
# `phoneme_sequence`: ditto for phonemes--with repeats
#
# Then saving them as tf_records


# Only need to change the following 3 lines
tfrecord_dir = '/home/bayuan/Documents/fall23/ecog2vec/elu_tl_models/tl_401_402/tfrecords_402_tl_on_401' # CHANGE TFRECORDS PATH HERE
patient = 'EFC402'    # CHANGE PER SUBJECT      
blocks = [4, 5, 6, 7, 8, 9, 13, 14, 15, 16, 17, 18, 19, 25, 26, 27, 33, 34, 35, 44, 45, 46, 47, 48, 49, 58, 59, 60]


# [4, 7, 10, 13, 19, 20, 21, 28, 35, 39, 52, 53, 54, 55, 56, 59, 60, 61, 62, 63, 64, 70, 73, 74, 75, 76, 77, 83, 92, 93, 94, 95, 97, 98, 99, 100, 101, 108, 109, 110, 111, 112, 113, 114, 115]

# [3, 4, 6, 7, 9, 10, 12, 13, 15, 17, 
#     18, 19, 20, 21, 22, 27, 28, 30, 33,
#     35, 38, 39, 40, 42, 44, 46, 48, 50,
#     52, 53, 54, 55, 56, 59, 60, 61, 62,
#     63, 64, 65, 70, 73, 74, 75, 76, 77,
#     83, 92, 93, 94, 95, 97, 98, 99, 100,
#     101, 108, 109, 110, 111, 112, 113, 
#     114, 115]

# [4, 5, 6, 7, 8, 9, 13, 14, 15, 16, 17, 18, 19, 25, 26, 27, 33, 34, 35, 44, 45, 46, 47, 48, 49, 58, 59, 60]

# [3, 23, 72]

# [4, 41, 57, 61, 66, 69, 73, 77, 83, 87]

def transcription_to_array(trial_t0, trial_tF, onset_times, offset_times, transcription, max_length, sampling_rate):
    
    # if the transcription is missing (e.g. for covert trials)
    if transcription is None:
        return np.full(max_length, 'pau', dtype='<U5')

    # get just the parts of transcript relevant to this trial
    trial_inds = (onset_times >= trial_t0) * (offset_times < trial_tF)
    transcript = np.array(transcription.description.split(' '))[trial_inds]
    onset_times = np.array(onset_times[trial_inds])
    offset_times = np.array(offset_times[trial_inds])

    # vectorized indexing
    sample_times = trial_t0 + np.arange(max_length)/sampling_rate
    indices = (
        (sample_times[None, :] >= onset_times[:, None]) *
        (sample_times[None, :] < offset_times[:, None])
    )

    # no more than one phoneme should be on at once...
    try:
        # print('exactly one phoneme:', np.all(np.sum(indices, 0) == 1))
        assert np.all(np.sum(indices, 0) < 2)
    except:
        pdb.set_trace()

    # ...but there can be locations with *zero* phonemes; assume 'pau' here
    transcript = np.insert(transcript, 0, 'pau')
    indices = np.sum(indices*(np.arange(1, len(transcript))[:, None]), 0)

    return transcript[indices]

def sentence_tokenize(token_list): # token_type = word_sequence
    tokenized_sentence = [
                (token.lower() + '_').encode('utf-8') for token in token_list
            ]
    return tokenized_sentence

def write_to_Protobuf(path, example_dicts):
    '''
    Collect the relevant ECoG data and then write to disk as a (google)
        protocol buffer.
    '''
    writer = tf.io.TFRecordWriter(
        path)
    for example_dict in example_dicts:
        feature_example = tfh.make_feature_example(example_dict)
        writer.write(feature_example.SerializeToString())
            
# sorting function for latent representation filenames
def custom_sort_key(filename):
    num_part = int(filename.split('nwb_')[1].split('.wav.pt')[0])
    return num_part

all_example_dict = []


for block in blocks:
    tfrecord_path = f'{tfrecord_dir}/{patient}_B{block}.tfrecord'

    nwb_filepath = folder_path = f"/NWB/{patient}/{patient}_B{block}.nwb"
    io = NWBHDF5IO(nwb_filepath, load_namespaces=True, mode='r')
    nwbfile = io.read()
    
    ### GETTING LATENT REPRESENTATION PATHS ###
    c_vectors_dir = '/home/bayuan/Documents/fall23/ecog2vec/wav2vec_outputs/latent_sentence'
    prefix = f'{patient}_B{block}.nwb_' 

    c_file_path_list = []

    for filename in os.listdir(c_vectors_dir):
        if filename.startswith(prefix):
            c_file_path_list.append(os.path.join(c_vectors_dir, filename))
            
    c_file_path_list = sorted(c_file_path_list, key=custom_sort_key)
    print(c_file_path_list)
    ###########################################
    phoneme_transcriptions = nwbfile.processing['behavior'].data_interfaces['BehavioralEpochs'].interval_series #['phoneme transcription'].timestamps[:]

    token_type = 'word_sequence'

    max_seconds_dict = {
        'phoneme': 0.2,
        'word': 1.0,
        'word_sequence': 6.25,
        'word_piece_sequence': 6.25,
        'phoneme_sequence': 6.25,
        'trial': 6.25
    }

    if 'phoneme transcription' in phoneme_transcriptions:
        print(f'Phoneme transcription for block {block} exists.')
        phoneme_transcript = phoneme_transcriptions['phoneme transcription']
        phoneme_onset_times = phoneme_transcript.timestamps[
            phoneme_transcript.data[()] == 1]
        phoneme_offset_times = phoneme_transcript.timestamps[
            phoneme_transcript.data[()] == -1]
    else:
        phoneme_transcript = None
        phoneme_onset_times = None
        phoneme_offset_times = None

    example_dicts = []

    makin_sr = 200 # screw it, actually just pass in c_sr everywhere
    c_sr = 101.7 # 399# 18

    for index, trial in enumerate(nwbfile.trials):
        
        # ECOG (C) SEQUENCE
        c_filepath = c_file_path_list[index]
        c = torch.load(c_filepath) #.detach().numpy()[0] # [1, nchannel, samples]
        # c = c.reshape(512,-1) # [nchannel, samples]
        c = c.T # [samples, nchannel]
    
        print(c.shape)
        nsamples = c.shape[0]
        
        # TEXT SEQUENCE
        speech_string = trial['transcription'].values[0]
        text_sequence = sentence_tokenize(speech_string.split(' ')) # , 'text_sequence')
        
        # AUDIO SEQUENCE    
        audio_sequence = []
        
        # PHONEME SEQUENCE
        t0 = float(trial.iloc[0].start_time)
        tF = float(trial.iloc[0].stop_time)
    
        i0 = np.rint(c_sr*t0).astype(int)
        iF = np.rint(c_sr*tF).astype(int)
        
        M = iF - i0
        
        max_seconds = max_seconds_dict.get(token_type) # , 0.2) # i don't think this 0.2 default is necessary for the scope of this
        max_samples = int(np.floor(c_sr*max_seconds))
        max_length = min(M, max_samples)
        
        phoneme_array = transcription_to_array(
                        t0, tF, phoneme_onset_times, phoneme_offset_times,
                        phoneme_transcript, max_length, c_sr # makin_sr
                    )
        
        phoneme_sequence = [ph.encode('utf-8') for ph in phoneme_array]
        
        if len(phoneme_sequence) != nsamples:
            if len(phoneme_sequence) > nsamples:
                phoneme_sequence = [phoneme_sequence[i] for i in range(nsamples)]
            else:
                for i in range(nsamples - len(phoneme_sequence)):
                    phoneme_sequence.append(phoneme_sequence[-1])
        
        print('\n------------------------')
        print(f'For sentence {index}: ')
        print(c[0:5,0:5])
        print(f'Latent representation shape: {c.shape} (should be [samples, nchannel])')
        print(text_sequence)
        print(f'Audio sequence: {audio_sequence}')
        print(f'Length of phoneme sequence: {len(phoneme_sequence)}')
        print(phoneme_sequence)
        print('------------------------\n')
        
        example_dicts.append({'ecog_sequence': c, 'text_sequence': text_sequence, 'audio_sequence': [], 'phoneme_sequence': phoneme_sequence,})
    #     break
    # break
    all_example_dict.extend(example_dicts)
    print(len(example_dicts))
    print(len(all_example_dict))
    write_to_Protobuf(tfrecord_path, example_dicts)

print(len(all_example_dict))
